# Tratamento dos Dados

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('../source/MICRODADOS_20201101.csv', sep = ';', encoding = 'cp1252')
df.head()

,DataNotificacao,DataCadastro,DataDiagnostico,DataColeta_RT_PCR,DataColetaTesteRapido,DataColetaSorologia,DataColetaSorologiaIGG,DataEncerramento,DataObito,Classificacao,...,FicouInternado,ViagemBrasil,ViagemInternacional,ProfissionalSaude,PossuiDeficiencia,MoradorDeRua,ResultadoRT_PCR,ResultadoTesteRapido,ResultadoSorologia,ResultadoSorologia_IGG
0,2020-10-31,2020-10-31,2020-10-30,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
1,2020-10-31,2020-10-31,2020-10-13,2020-10-20,NaN,NaN,NaN,NaN,NaN,Descartados,...,Não Informado,Não,Não,Não,Não,Não,Negativo,Não Informado,Não Informado,Não Informado
2,2020-10-31,2020-10-31,2020-10-24,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não Informado,Não Informado,Não Informado,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
3,2020-10-31,2020-10-31,2020-10-25,NaN,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado
4,2020-10-31,2020-10-31,2020-10-25,2020-10-31,NaN,NaN,NaN,NaN,NaN,Suspeito,...,Não,Não,Não,Não,Não,Não,Não Informado,Não Informado,Não Informado,Não Informado


In [6]:
print('Classificacão: {}'.format(df['Classificacao'].unique()))
print('Evolução: {}'.format(df['Evolucao'].unique()))

Classificacão: ['Suspeito' 'Descartados' 'Confirmados']
Evolução: ['-' 'Ignorado' 'Cura' 'Óbito pelo COVID-19' 'Óbito por outras causas']


In [7]:
total_casos = df.query('Classificacao == "Confirmados"').shape[0]
total_obitos = df.query('Evolucao == "Óbito pelo COVID-19"').shape[0]
total_curas = df.query('Evolucao == "Cura"').shape[0]

print('TOTAIS')
print('=================')
print('  - CASOS: {}'.format(total_casos))
print('  - ÓBITOS: {} ({}%)'.format(total_obitos, round((total_obitos * 100) / total_casos, 1)))
print('  - CURA: {} ({}%)'.format(total_curas, round((total_curas * 100) / total_casos, 1)))

TOTAIS
  - CASOS: 155721
  - ÓBITOS: 3855 (2.5%)
  - CURA: 145951 (93.7%)


In [8]:
df['DataNotificacao'] = pd.to_datetime(df['DataNotificacao'])
df.sort_values('DataNotificacao', inplace=True)
df = df.query('Classificacao == "Confirmados"').reset_index(drop=True)

## Cálculo de Confirmados

In [9]:
df['ConfirmadosAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumcount() + 1

df['Confirmados'] = df[['Municipio', 'Bairro', 'DataNotificacao']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumcount() + 1

## Cálculo de Óbitos

In [10]:
df['Obitos'] = df['Evolucao'].apply(lambda evolucao: 1 if evolucao == 'Óbito pelo COVID-19' else 0)

df['ObitosAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum() + 1

df['Obitos'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Obitos']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumsum() + 1

## Cálculo de Curas

In [11]:
df['Curas'] = df['Evolucao'].apply(lambda evolucao: 1 if evolucao == 'Cura' else 0)

df['CurasAcumulado'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro'])\
    .cumsum() + 1

df['Curas'] = df[['Municipio', 'Bairro', 'DataNotificacao', 'Curas']]\
    .groupby(['Municipio', 'Bairro', 'DataNotificacao'])\
    .cumsum() + 1

## Contagem de Casos, Óbitos e Curas

In [12]:
grupo_base = ['DataNotificacao', 'Municipio', 'Bairro']

datas = df[['DataNotificacao']].drop_duplicates().reset_index(drop=True)
municipios_bairros = df[['Municipio', 'Bairro']].drop_duplicates().reset_index(drop=True)

datas['key'] = 0
municipios_bairros['key'] = 0

In [13]:
df_counts = pd.merge(datas, municipios_bairros, how='outer')[grupo_base]

df_counts = df_counts.merge(
    df[[
        'Municipio',
        'Bairro',
        'DataNotificacao',
        'Confirmados',
        'Obitos',
        'Curas',
        'ConfirmadosAcumulado',
        'ObitosAcumulado',
        'CurasAcumulado'
    ]].drop_duplicates(grupo_base, keep='last')\
        .dropna(),
    on=grupo_base,
    how='left'
)

df_counts.fillna({
    'Confirmados': 0,
    'Obitos': 0,
    'Curas': 0,
})

columns_ffill = ['ConfirmadosAcumulado', 'ObitosAcumulado', 'CurasAcumulado']

df_counts[columns_ffill] = df_counts.groupby(['Municipio', 'Bairro'])\
    .fillna(method='ffill')\
    .fillna(0)\
    [columns_ffill]

In [14]:
df_counts.head()

,DataNotificacao,Municipio,Bairro,Confirmados,Obitos,Curas,ConfirmadosAcumulado,ObitosAcumulado,CurasAcumulado
0,2020-02-29,VILA VELHA,ITAPUÃ,1.0,1.0,2.0,1.0,1.0,2.0
1,2020-02-29,VILA VELHA,PRAIA DE ITAPARICA,NaN,NaN,NaN,0.0,0.0,0.0
2,2020-02-29,VITORIA,JARDIM DA PENHA,NaN,NaN,NaN,0.0,0.0,0.0
3,2020-02-29,LINHARES,INTERLAGOS,NaN,NaN,NaN,0.0,0.0,0.0
4,2020-02-29,VILA VELHA,PRAIA DAS GAIVOTAS,NaN,NaN,NaN,0.0,0.0,0.0


In [15]:
df_counts['Bairro'].unique()

array(['ITAPUÃ', 'PRAIA DE ITAPARICA', 'JARDIM DA PENHA', ...,
       'CÓRREGO SANTA ROSA DO MUNIZ', 'SANTA LUZIA PICADÃO',
       'SÃO DALMÁCIO'], dtype=object)

In [16]:
from unidecode import unidecode

In [17]:
df_counts['Municipio'] = df_counts['Municipio'].apply(lambda x: unidecode(str(x)).upper())
df_counts['Bairro'] = df_counts['Bairro'].apply(lambda x: unidecode(str(x)).upper())

In [19]:
df_counts.to_csv('../notebooks_output/microdados_pre-processed.csv', sep=',', index=False, encoding='UTF-8')

# Visualização de Dados

In [60]:
import plotly.express as px

In [49]:
df_bar = df_counts.copy()

fig = px.line(
    df_counts.groupby(['DataNotificacao', 'Municipio'])
        .sum()
        .reset_index()
        .query('Municipio == "VITORIA" | Municipio == "SERRA"'),
    x="DataNotificacao",
    y="Obitos",
    color="Municipio",
    hover_name="Municipio",
    labels={
        'DataNotificacao': 'Data',
        'Obitos': 'Óbitos',
        'Municipio': 'Município'
    }
)

fig.show()